# Embeddings
* Transform the small parts of text in numbers (vectors) that are easily stored and searched by vector databases.

In [ ]:
#pip install python-dotenv

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain-openai

* NOTE: Since right now is the best LLM in the market, we will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral in a next lesson.

In [2]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-3.5-turbo-0125")

## Reminder: Steps of the RAG process.
* When you load a document, you end up with strings. Sometimes the strings will be too large to fit into the context window. In those occassions we will use the RAG technique:
    * Split document in small chunks.
    * **Transform text chunks in numeric chunks (embeddings)**.
    * Load embeddings to a vector database (aka vector store).
    * Load question and retrieve the most relevant embeddings to respond it.
    * Sent the embeddings to the LLM to format the response properly.

## Splitters: divide the loaded document in small chunks of text
* Also called "Document Transformers".
* See the documentation page [here](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/).
* See the list of built-in splitters [here](https://python.langchain.com/v0.1/docs/integrations/document_transformers/).

#### Simple splitting by character

In [3]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./data/be-good.txt")

loaded_data = loader.load()

In [4]:
loaded_data

[Document(metadata={'source': './data/be-good.txt'}, page_content='Be good\n\nApril 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the\nphrase that became our motto: Make something people want.  We\'ve\nlearned a lot since then, but if I were choosing now that\'s still\nthe one I\'d pick.Another thing we tell founders is not to worry too much about the\nbusiness model, at least at first.  Not because making money is\nunimportant, but because it\'s so much easier than building something\ngreat.A couple weeks ago I realized that if you put those two ideas\ntogether, you get something surprising.  Make something people want.\nDon\'t worry too much about making money.  What you\'ve got is a\ndescription of a charity.When you get an unexpected result like this, it could either be a\nbug or a new discovery.  Either businesses aren\'t supposed to be\nlike charities, and we\'ve proven by reductio ad absurdum that o

In [5]:
loaded_data[0].page_content

'Be good\n\nApril 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the\nphrase that became our motto: Make something people want.  We\'ve\nlearned a lot since then, but if I were choosing now that\'s still\nthe one I\'d pick.Another thing we tell founders is not to worry too much about the\nbusiness model, at least at first.  Not because making money is\nunimportant, but because it\'s so much easier than building something\ngreat.A couple weeks ago I realized that if you put those two ideas\ntogether, you get something surprising.  Make something people want.\nDon\'t worry too much about making money.  What you\'ve got is a\ndescription of a charity.When you get an unexpected result like this, it could either be a\nbug or a new discovery.  Either businesses aren\'t supposed to be\nlike charities, and we\'ve proven by reductio ad absurdum that one\nor both of the principles we began with is false.  Or we have 

In [6]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [7]:
texts = text_splitter.create_documents([loaded_data[0].page_content])

In [8]:
len(texts)

2

In [9]:
texts[0]

Document(page_content='Be good')

In [11]:
texts[1]

Document(page_content='April 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the\nphrase that became our motto: Make something people want.  We\'ve\nlearned a lot since then, but if I were choosing now that\'s still\nthe one I\'d pick.Another thing we tell founders is not to worry too much about the\nbusiness model, at least at first.  Not because making money is\nunimportant, but because it\'s so much easier than building something\ngreat.A couple weeks ago I realized that if you put those two ideas\ntogether, you get something surprising.  Make something people want.\nDon\'t worry too much about making money.  What you\'ve got is a\ndescription of a charity.When you get an unexpected result like this, it could either be a\nbug or a new discovery.  Either businesses aren\'t supposed to be\nlike charities, and we\'ve proven by reductio ad absurdum that one\nor both of the principles we began with is false.  

#### Splitting with metadata

In [55]:
metadatas = [{"chunk": 0}, {"chunk": 1}]

documents = text_splitter.create_documents(
    [loaded_data[0].page_content, loaded_data[0].page_content], 
    metadatas=metadatas
)

In [58]:
documents[0]

Document(page_content='Be good', metadata={'chunk': 0})

In [59]:
print(documents[0])

page_content='Be good' metadata={'chunk': 0}


## Embeddings: transform chunks of text in chunks of numbers
* Vector databases work very fast with numbers.
* See the documentation page [here](https://python.langchain.com/v0.1/docs/modules/data_connection/text_embedding/).
* See the list of embedding model providers [here](https://python.langchain.com/v0.1/docs/integrations/text_embedding/).
* **The base Embeddings class in LangChain provides two methods: one for embedding documents and one for embedding a query**.

In [12]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [13]:
chunks_of_text =     [
        "Hi there!",
        "Hello!",
        "What's your name?",
        "Bond, James Bond",
        "Hello Bond!"
    ]

In [14]:
embeddings = embeddings_model.embed_documents(chunks_of_text)

In [17]:
# embeddings

In [18]:
len(embeddings)

5

In [19]:
len(embeddings[0])

1536

In [20]:
print(embeddings[0][:5])

[-0.020350975915789604, -0.007109511643648148, -0.022864697501063347, -0.026229018345475197, -0.0374772809445858]


In [21]:
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")

In [22]:
len(embedded_query)

1536

## hugging face embeddings

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm import tqdm

embeddings_model_name = 'BAAI/bge-large-en'

In [2]:
chunks_of_text =     [
        "Hi there!",
        "Hello!",
        "What's your name?",
        "Bond, James Bond",
        "Hello Bond!"
    ]

In [10]:
dir(HuggingFaceEmbeddings)

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '__weakref__',
 '_abc_impl',
 '_calculate_keys',
 '_copy_and_set_values',
 '_decompose

In [4]:
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)

c:\Users\kumar\.conda\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\kumar\.conda\envs\myenv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kumar\.cache\huggingface\hub\models--BAAI--bge-large-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode

In [5]:
embed=embeddings.embed_documents(chunks_of_text)

In [6]:
embed[0]

[0.022317636758089066,
 0.0203997939825058,
 0.0014153452357277274,
 0.02069493941962719,
 -0.020114384591579437,
 0.004469053819775581,
 -0.011439832858741283,
 0.03229910135269165,
 0.024258386343717575,
 0.01829921081662178,
 0.02409200742840767,
 0.004192252177745104,
 -0.005777841433882713,
 -0.0008624229812994599,
 -0.029132386669516563,
 0.02365010976791382,
 0.0034695384092628956,
 -0.03188753128051758,
 -0.019553950056433678,
 -0.0035717368591576815,
 -0.004739687312394381,
 -0.01060212031006813,
 -0.06562972068786621,
 0.0031894254498183727,
 0.0026779670733958483,
 0.023825008422136307,
 0.030809486284852028,
 0.0008644777117297053,
 0.057766519486904144,
 0.03842684626579285,
 -0.030902331694960594,
 -0.02830774523317814,
 0.022647976875305176,
 -0.0429609939455986,
 -0.04301609471440315,
 -0.015443487092852592,
 0.013308895751833916,
 -0.06171681359410286,
 -0.012224046513438225,
 -0.034827087074518204,
 0.03101651929318905,
 0.005374361760914326,
 0.053475622087717056,
 -

In [15]:
len(embed)

5

In [7]:
embed[0][:3]

[0.022317636758089066, 0.0203997939825058, 0.0014153452357277274]

In [13]:
embedded_query = embeddings.embed_query("What was the name mentioned in the conversation?")

In [14]:
embedded_query

[0.019107665866613388,
 -0.013160187751054764,
 -0.02585792914032936,
 0.02108672820031643,
 -0.026419661939144135,
 -0.03060021810233593,
 -0.0025607424322515726,
 0.030160995200276375,
 0.01421373337507248,
 -0.008128867484629154,
 0.06770917773246765,
 0.007823786698281765,
 -0.011056212708353996,
 0.003085013013333082,
 -0.03223785012960434,
 0.007320853415876627,
 -0.020412813872098923,
 -0.028189782053232193,
 -0.038000185042619705,
 -0.024629589170217514,
 0.02178928442299366,
 0.019319111481308937,
 -0.06808235496282578,
 0.020841268822550774,
 -0.014210849069058895,
 0.006465389393270016,
 -0.011744367890059948,
 -0.012168467976152897,
 0.05784671753644943,
 0.07606912404298782,
 -0.02949383668601513,
 -0.02006453648209572,
 0.03098880499601364,
 -0.027962731197476387,
 -0.02690696157515049,
 -0.05155469477176666,
 0.02923855185508728,
 -0.04615291208028793,
 -0.047649431973695755,
 -0.04002062603831291,
 -0.018223222345113754,
 -0.016508813947439194,
 0.03649435564875603,
 -0

## ollama embeddings

In [ ]:
!pip install -qU langchain-ollama

In [33]:
!pip install ollama

In [40]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama3.2",
)

In [41]:
chunks_of_text =     [
        "Hi there!",
        "Hello!",
        "What's your name?",
        "Bond, James Bond",
        "Hello Bond!"
    ]

In [42]:
embedded_query = embeddings.embed_query("What was the name mentioned in the conversation?")

In [45]:
len(embedded_query)

3072

In [43]:
embedded_query

[0.00033650864,
 -0.0008525989,
 -0.031480964,
 -0.025903516,
 0.021391168,
 -0.0031938923,
 0.014638866,
 0.007367054,
 -0.0138093205,
 0.011606522,
 0.0093392655,
 0.016804568,
 0.021747477,
 0.020545252,
 -0.010547544,
 -0.016376458,
 0.011131985,
 -0.0024259302,
 -0.012963058,
 -0.019700155,
 -0.008959243,
 -0.002068587,
 0.019353004,
 -0.0098124165,
 -0.011438436,
 0.013965733,
 0.011893537,
 -0.013277909,
 0.017502109,
 0.0122880265,
 -0.0042248755,
 0.017611539,
 -0.011659668,
 -0.009839821,
 0.003087168,
 -0.005020855,
 0.007222036,
 -0.0016693082,
 -0.0034489133,
 0.00954479,
 -0.012288966,
 0.016271193,
 -0.0014082452,
 -0.0067253346,
 -0.015811384,
 -0.011142137,
 -0.00027863856,
 0.010321822,
 0.0067968746,
 -0.0015908865,
 0.024269266,
 0.007929096,
 -0.012700699,
 0.036691934,
 0.0074847704,
 -0.0199964,
 0.018790495,
 -0.047085106,
 0.0027912986,
 0.012476727,
 0.0081553245,
 -0.004604222,
 -0.020010252,
 -0.030101184,
 0.011220197,
 -0.043934964,
 0.013080675,
 0.005649

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 003-embeddings.py
* In terminal, make sure you are in the directory of the file and run:
    * python 003-embeddings.py